In [1]:
import torch
import numpy as np
import os
import glob
from brainscore_vision import load_model

/home/seo/miniconda3/envs/myvenv/lib/python3.11/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


  Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached albumentations-1.4.22-py3-none-any.whl.metadata (33 kB)
  Using cached timm-1.0.12-py3-none-any.whl.metadata (51 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached albucore-0.0.21-py3-none-any.whl.metadata (5.3 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached stringzilla-3.11.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl.metadata (80 kB)
  Using cached simsimd-6.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (66 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metad

In [ ]:
model = load_model('deit_large_imagenet_full_seed-0')

In [3]:
act_model = model.activations_model

In [4]:
normalizations_layer = []
for i in act_model.layers():
    if type(i[1]).__name__ == 'LayerNorm':
        # print(type(i[1]).__name__)
        # print(i)
        normalizations_layer += [i[0]]

In [4]:
normalizations_layer

['blocks.0.norm1',
 'blocks.0.norm2',
 'blocks.1.norm1',
 'blocks.1.norm2',
 'blocks.2.norm1',
 'blocks.2.norm2',
 'blocks.3.norm1',
 'blocks.3.norm2',
 'blocks.4.norm1',
 'blocks.4.norm2',
 'blocks.5.norm1',
 'blocks.5.norm2',
 'blocks.6.norm1',
 'blocks.6.norm2',
 'blocks.7.norm1',
 'blocks.7.norm2',
 'blocks.8.norm1',
 'blocks.8.norm2',
 'blocks.9.norm1',
 'blocks.9.norm2',
 'blocks.10.norm1',
 'blocks.10.norm2',
 'blocks.11.norm1',
 'blocks.11.norm2',
 'blocks.12.norm1',
 'blocks.12.norm2',
 'blocks.13.norm1',
 'blocks.13.norm2',
 'blocks.14.norm1',
 'blocks.14.norm2',
 'blocks.15.norm1',
 'blocks.15.norm2',
 'blocks.16.norm1',
 'blocks.16.norm2',
 'blocks.17.norm1',
 'blocks.17.norm2',
 'blocks.18.norm1',
 'blocks.18.norm2',
 'blocks.19.norm1',
 'blocks.19.norm2',
 'blocks.20.norm1',
 'blocks.20.norm2',
 'blocks.21.norm1',
 'blocks.21.norm2',
 'blocks.22.norm1',
 'blocks.22.norm2',
 'blocks.23.norm1',
 'blocks.23.norm2',
 'norm']

In [ ]:
imagenet = f'{ os.getcwd() }/data/imagenet/val'

In [5]:
stimuli = glob.glob(os.path.join(imagenet, '**'), recursive=True)
stimuli = [ stimulus for stimulus in stimuli if os.path.isfile(stimulus) ]

In [ ]:
output = f'{ os.getcwd() }/data/output/deit'

In [7]:
# Set batch size for processing
batch_size = 1000

# Initialize counter for file naming
counter = 1

# Loop through the stimuli in batches
for batch_start in range(0, len(stimuli), batch_size):
    # Get the current batch of stimuli
    batch = stimuli[batch_start:batch_start + batch_size]

    # Extract model activations for the batch
    model_extraction = act_model(batch, normalizations_layer)

    # Get the list of layers
    layers = model_extraction['layer'].values

    # Extract features by layer for the entire batch
    features_by_layer = {
        layer: model_extraction.sel(neuroid=model_extraction['layer'] == layer).data
        for layer in normalizations_layer
      }

    # Create a dictionary to save in .npz format
    npz_data = {
        layer_name: features 
        for layer_name, features in features_by_layer.items()
      }
    # Save the data for the current batch
    np.savez(os.path.join(output, '{}-{}.npz'.format(model.identifier, counter)), **npz_data)

    # Increment the counter
    counter += 1

activations:   0%|          | 0/1024 [00:00<?, ?it/s]

: 

In [ ]:
# # How to make flatten version (2D)

# # Get the list of layers
# layers = model_extraction['layer'].values

# # Extract features by layer
# features_by_layer = {layer: model_extraction.sel(neuroid=model_extraction['layer'] == layer).data for layer in conv_list}

In [ ]:
# # Create a dictionary to save in .npz format
# npz_data = {layer_name: features for layer_name, features in features_by_layer.items()}

# # Save as an npz file
# np.savez('{}.npz'.format(model.identifier), **npz_data)

In [ ]:
# import os

# # Create a dictionary to save in .npz format
# npz_data = {layer_name: features for layer_name, features in features_by_layer.items()}

# # path
# path1 = '/home/seo/extracted_data'
# path2 = '/home/remmal/imagenet/extracted_data'

# # Save as an npz file
# np.savez(os.path.join(path1, '{}.npz'.format(model.identifier)), **npz_data)
# np.savez(os.path.join(path2, '{}.npz'.format(model.identifier)), **npz_data)

# print("Files saved in both locations.")
